In [114]:
import os, sys
from os.path import isdir, isfile
import os
import sys
import argparse
import csv
import pandas as pd
import datetime as dt
import numpy as np
from pprint import pprint
from tqdm import tqdm
from pytictoc import TicToc
import traceback
from statistics import median
from statistics import mean
from statistics import mode
from statistics import stdev
from scipy import stats
from scipy import signal
import portion as P
import math
import random
import itertools as it

pd.set_option('display.max_columns', 200)
# pd.set_option('display.max_rows', 200)

def makedir(dirpath, mode=0):  # mode=1: show message, mode=0: hide message
    if os.path.isdir(dirpath):
        if mode:
            print("mkdir: cannot create directory '{}': directory has already existed.".format(dirpath))
        return
    ### recursively make directory
    _temp = []
    while not os.path.isdir(dirpath):
        _temp.append(dirpath)
        dirpath = os.path.dirname(dirpath)
    while _temp:
        dirpath = _temp.pop()
        print("mkdir", dirpath)
        os.mkdir(dirpath)

def dfs(visited, node, filetags, level=0):  # function for dfs
    global lofile
    if node in visited:
        return
    visited.add(node)
    ### TODO: 'node' is an absolute pathname (string) for a file or a directory.
    if os.path.basename(node).startswith(filetags):
        lofile.append(node)
    ### TODO END
    if isfile(node):
        return
    children = [os.path.join(node, s) for s in sorted(os.listdir(node))]
    for child in children:
        dfs(visited, child, filetags, level+1)
    ### print message in the end
    if level == 0:
        # print("**********************************************************************")
        pass

# Collect Files

In [115]:
ls_dnlk = []
ls_uplk = []

root = "/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04"
lofile = ls_dnlk
visited = set()  # Set to keep track of visited nodes of graph (tree).
dfs(visited, root, filetags=('udp_dnlk_loss_latency_ho.pkl'))
ls_dnlk = lofile

lofile = ls_uplk
visited = set()
dfs(visited, root, filetags=('udp_uplk_loss_latency_ho.pkl'))
ls_uplk = lofile

root = "/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04#1"
lofile = ls_dnlk
visited = set()  # Set to keep track of visited nodes of graph (tree).
dfs(visited, root, filetags=('udp_dnlk_loss_latency_ho.pkl'))
ls_dnlk = lofile

lofile = ls_uplk
visited = set()
dfs(visited, root, filetags=('udp_uplk_loss_latency_ho.pkl'))
ls_uplk = lofile


ls_dnlk_qc01 = [s for s in ls_dnlk if 'qc01' in s]
ls_dnlk_qc02 = [s for s in ls_dnlk if 'qc02' in s]
ls_dnlk_qc03 = [s for s in ls_dnlk if 'qc03' in s]
ls_dnlk = list(zip(ls_dnlk_qc01, ls_dnlk_qc02, ls_dnlk_qc03))

ls_uplk_qc01 = [s for s in ls_uplk if 'qc01' in s]
ls_uplk_qc02 = [s for s in ls_uplk if 'qc02' in s]
ls_uplk_qc03 = [s for s in ls_uplk if 'qc03' in s]
ls_uplk = list(zip(ls_uplk_qc01, ls_uplk_qc02, ls_uplk_qc03))

pprint(ls_dnlk)
pprint(ls_uplk)

[('/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04/_Bandlock_Udp_B3_B7_B8_RM500Q/qc01/#01/data/udp_dnlk_loss_latency_ho.pkl',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04/_Bandlock_Udp_B3_B7_B8_RM500Q/qc02/#01/data/udp_dnlk_loss_latency_ho.pkl',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04/_Bandlock_Udp_B3_B7_B8_RM500Q/qc03/#01/data/udp_dnlk_loss_latency_ho.pkl'),
 ('/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04/_Bandlock_Udp_B3_B7_B8_RM500Q/qc01/#02/data/udp_dnlk_loss_latency_ho.pkl',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04/_Bandlock_Udp_B3_B7_B8_RM500Q/qc02/#02/data/udp_dnlk_loss_latency_ho.pkl',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04/_Bandlock_Udp_B3_B7_B8_RM500Q/qc03/#02/data/udp_dnlk_loss_latency_ho.pkl'),
 ('/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04#1/_Bandlock_Udp_B3_B7_B8_RM500Q/qc01/#01/data/udp_dnlk_loss_latency_ho.pkl',
  '/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04#1/_Bandlock_Udp_B3_B7_B

# Start

In [116]:
sorter = ['stable',
          'LTE_HO','MN_HO','SN_HO','MNSN_HO','SN_Rel','SN_Setup','SN_Rel_MN_HO','SN_Setup_MN_HO',
          'SCG_Failure_t310-Expiry (0)','SCG_Failure_randomAccessProblem (1)','SCG_Failure_rlc-MaxNumRetx (2)','SCG_Failure_synchReconfigFailureSCG (3)',
          'SCG_Failure_scg-ReconfigFailure (4)','SCG_Failure_srb3-IntegrityFailure (5)','SCG_Failure_other-r16 (6)',
          'MCG_Failure_reconfigurationFailure (0)','MCG_Failure_handoverFailure (1)','MCG_Failure_otherFailure (2)'
          'NAS_Recovery_reconfigurationFailure (0)','NAS_Recovery_handoverFailure (1)','NAS_Recovery_otherFailure (2)']
# mcg failure types: https://www.sharetechnote.com/html/Handbook_LTE_RRC_ConnectionReestablishment.html (0)(1)(2)
# scg failure types: https://www.sharetechnote.com/html/5G/5G_Release17.html (1)(3)

def align_data(dfs):
    st, et = [], []
    for schm in dfs.keys():
        st.append(dfs[schm]['seq'].array[0])
        et.append(dfs[schm]['seq'].array[-1])
    st, et = max(st), min(et)
    for schm in dfs.keys():
        dfs[schm] = dfs[schm][(dfs[schm]['seq'] >= st) & (dfs[schm]['seq'] <= et)].reset_index(drop=True)
        dfs[schm]['ho_type0'] = dfs[schm]['ho_type0'].cat.set_categories(sorter)
        # for col in dfs[schm].columns[2:]:
        #     dfs[schm] = dfs[schm].rename(columns={col:f'{col}_{schm}'})
    return dfs

def consolidate_data(dfs, schemes=schemes):
    xs = list(it.combinations(range(len(schemes)), 2))
    print(xs)
    xs = [(schemes[s1], schemes[s2]) for (s1, s2) in xs]
    print(xs)
    for schm1, schm2 in xs:
        df1, df2 = dfs[schm1].copy(), dfs[schm2].copy()
        key = f'{schm1}+{schm2}'
        print(key)
        # dfs[key] = pd.merge(df1, df2, on=['seq','Timestamp'])
        # dfs[key] = pd.merge(df1, df2, on=['seq','Timestamp'], suffixes=(f'_{schm1}', f'_{schm2}'))
        dfs[key] = pd.merge(df1, df2, on=['seq','Timestamp'], suffixes=(f'_{schm1}', f'_{schm2}'), how='outer')
        dfs[key][f'lost_{key}'] = dfs[key][f'lost_{schm1}'] & dfs[key][f'lost_{schm2}']
        dfs[key][f'lost_{schm1}+^{schm2}'] = dfs[key][f'lost_{schm1}'] & ~dfs[key][f'lost_{schm2}']
        dfs[key][f'lost_^{schm1}+{schm2}'] = ~dfs[key][f'lost_{schm1}'] & dfs[key][f'lost_{schm2}']
        dfs[key][f'lost_^{schm1}+^{schm2}'] = ~(dfs[key][f'lost_{schm1}'] | dfs[key][f'lost_{schm2}'])
        dfs[key][f'excl_{key}'] = dfs[key][f'excl_{schm1}'] & dfs[key][f'excl_{schm2}']
        dfs[key][f'excl_{schm1}+^{schm2}'] = dfs[key][f'excl_{schm1}'] & ~dfs[key][f'excl_{schm2}']
        dfs[key][f'excl_^{schm1}+{schm2}'] = ~dfs[key][f'excl_{schm1}'] & dfs[key][f'excl_{schm2}']
        dfs[key][f'excl_^{schm1}+^{schm2}'] = ~(dfs[key][f'excl_{schm1}'] | dfs[key][f'excl_{schm2}'])
        dfs[key][f'latency_{key}'] = dfs[key][[f'latency_{schm1}', f'latency_{schm2}']].min(axis=1)
    return dfs

In [117]:
# define settings
devices = ['qc01','qc02','qc03']
schemes = ['B3','B7','B8']

dfs_dnlk = []

# read data
for testpath in ls_dnlk:
    trace = os.path.abspath(os.path.join(os.path.dirname(testpath[0]), '..'))[-3:]
    dirpath = os.path.abspath(os.path.join(os.path.dirname(testpath[0]), '..', '..', '..', 'combo', trace))
    print(dirpath)
    makedir(dirpath)

    dfs = {}
    for i, (dev, schm) in enumerate(zip(devices, schemes)):
        dfs[schm] = pd.read_pickle(testpath[i])

    # consolidate data & output
    dfs = align_data(dfs)
    dfs = consolidate_data(dfs)
    for schm, df in dfs.items():
        df.to_pickle(os.path.join(dirpath, f'udp_dnlk_loss_latency_ho_{schm}.pkl'))
        # display(df)
    dfs_dnlk.append(dfs)

    # testing
    xs = list(it.combinations(range(len(schemes)), 2))
    xs = [(schemes[s1], schemes[s2]) for (s1, s2) in xs]
    xsk = [f'{s1}+{s2}' for (s1, s2) in xs]
    print([*schemes, *xsk])

/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04/_Bandlock_Udp_B3_B7_B8_RM500Q/combo/#01
[(0, 1), (0, 2), (1, 2)]
[('B3', 'B7'), ('B3', 'B8'), ('B7', 'B8')]
B3+B7
B3+B8
B7+B8
['B3', 'B7', 'B8', 'B3+B7', 'B3+B8', 'B7+B8']
/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04/_Bandlock_Udp_B3_B7_B8_RM500Q/combo/#02
[(0, 1), (0, 2), (1, 2)]
[('B3', 'B7'), ('B3', 'B8'), ('B7', 'B8')]
B3+B7
B3+B8
B7+B8
['B3', 'B7', 'B8', 'B3+B7', 'B3+B8', 'B7+B8']
/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04#1/_Bandlock_Udp_B3_B7_B8_RM500Q/combo/#01
[(0, 1), (0, 2), (1, 2)]
[('B3', 'B7'), ('B3', 'B8'), ('B7', 'B8')]
B3+B7
B3+B8
B7+B8
['B3', 'B7', 'B8', 'B3+B7', 'B3+B8', 'B7+B8']
/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04#1/_Bandlock_Udp_B3_B7_B8_RM500Q/combo/#02
[(0, 1), (0, 2), (1, 2)]
[('B3', 'B7'), ('B3', 'B8'), ('B7', 'B8')]
B3+B7
B3+B8
B7+B8
['B3', 'B7', 'B8', 'B3+B7', 'B3+B8', 'B7+B8']


In [118]:
# define settings
devices = ['qc01','qc02','qc03']
schemes = ['B3','B7','B8']

dfs_uplk = []

# read data
for testpath in ls_uplk:
    trace = os.path.abspath(os.path.join(os.path.dirname(testpath[0]), '..'))[-3:]
    dirpath = os.path.abspath(os.path.join(os.path.dirname(testpath[0]), '..', '..', '..', 'combo', trace))
    print(dirpath)
    makedir(dirpath)

    dfs = {}
    for i, (dev, schm) in enumerate(zip(devices, schemes)):
        dfs[schm] = pd.read_pickle(testpath[i])

    # consolidate data & output
    dfs = align_data(dfs)
    dfs = consolidate_data(dfs)
    for schm, df in dfs.items():
        df.to_pickle(os.path.join(dirpath, f'udp_uplk_loss_latency_ho_{schm}.pkl'))
        # display(df)
    dfs_uplk.append(dfs)

    # testing
    xs = list(it.combinations(range(len(schemes)), 2))
    xs = [(schemes[s1], schemes[s2]) for (s1, s2) in xs]
    xsk = [f'{s1}+{s2}' for (s1, s2) in xs]
    print([*schemes, *xsk])

/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04/_Bandlock_Udp_B3_B7_B8_RM500Q/combo/#01
[(0, 1), (0, 2), (1, 2)]
[('B3', 'B7'), ('B3', 'B8'), ('B7', 'B8')]
B3+B7
B3+B8
B7+B8
['B3', 'B7', 'B8', 'B3+B7', 'B3+B8', 'B7+B8']
/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04/_Bandlock_Udp_B3_B7_B8_RM500Q/combo/#02
[(0, 1), (0, 2), (1, 2)]
[('B3', 'B7'), ('B3', 'B8'), ('B7', 'B8')]
B3+B7
B3+B8
B7+B8
['B3', 'B7', 'B8', 'B3+B7', 'B3+B8', 'B7+B8']
/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04#1/_Bandlock_Udp_B3_B7_B8_RM500Q/combo/#01
[(0, 1), (0, 2), (1, 2)]
[('B3', 'B7'), ('B3', 'B8'), ('B7', 'B8')]
B3+B7
B3+B8
B7+B8
['B3', 'B7', 'B8', 'B3+B7', 'B3+B8', 'B7+B8']
/Users/jackbedford/Desktop/MOXA/Code/data/2023-02-04#1/_Bandlock_Udp_B3_B7_B8_RM500Q/combo/#02
[(0, 1), (0, 2), (1, 2)]
[('B3', 'B7'), ('B3', 'B8'), ('B7', 'B8')]
B3+B7
B3+B8
B7+B8
['B3', 'B7', 'B8', 'B3+B7', 'B3+B8', 'B7+B8']


In [119]:
dfs_dnlk

[{'B3':            seq                  Timestamp   lost   excl   latency  \
  0        10541 2023-02-04 14:58:56.963738  False  False  0.016078   
  1        10542 2023-02-04 14:58:56.965738  False  False  0.020034   
  2        10543 2023-02-04 14:58:56.967738  False  False  0.018035   
  3        10544 2023-02-04 14:58:56.969738  False  False  0.016074   
  4        10545 2023-02-04 14:58:56.971738  False  False  0.021257   
  ...        ...                        ...    ...    ...       ...   
  225844  236385 2023-02-04 15:06:28.672845  False  False  0.006868   
  225845  236386 2023-02-04 15:06:28.674845  False  False  0.010810   
  225846  236387 2023-02-04 15:06:28.676845  False  False  0.008811   
  225847  236388 2023-02-04 15:06:28.678845  False  False  0.006812   
  225848  236389 2023-02-04 15:06:28.680845  False  False  0.004812   
  
                          xmit_time                   arr_time  ho_index  \
  0      2023-02-04 14:58:56.963746 2023-02-04 14:58:56.979816 

In [120]:
dfs_uplk

[{'B3':            seq                  Timestamp   lost   excl   latency  \
  0        10541 2023-02-04 14:58:56.969418  False  False  0.005397   
  1        10542 2023-02-04 14:58:56.971418  False  False  0.007972   
  2        10543 2023-02-04 14:58:56.973418  False  False  0.006414   
  3        10544 2023-02-04 14:58:56.975418  False  False  0.007028   
  4        10545 2023-02-04 14:58:56.977418  False  False  0.006989   
  ...        ...                        ...    ...    ...       ...   
  225844  236385 2023-02-04 15:06:28.666497  False  False  0.015948   
  225845  236386 2023-02-04 15:06:28.668497  False  False  0.015905   
  225846  236387 2023-02-04 15:06:28.670497  False  False  0.014362   
  225847  236388 2023-02-04 15:06:28.672497  False  False  0.016974   
  225848  236389 2023-02-04 15:06:28.674497  False  False  0.015350   
  
                          xmit_time                   arr_time  ho_index  \
  0      2023-02-04 14:58:56.969429 2023-02-04 14:58:56.974815 

In [121]:
for i in range(1, len(dfs_dnlk)):
    for schm in dfs_dnlk[0].keys():
        dfs_dnlk[0][schm] = pd.concat([dfs_dnlk[0][schm], dfs_dnlk[i][schm]], ignore_index=True)
dfs_dnlk = dfs_dnlk[0]

for i in range(1, len(dfs_uplk)):
    for schm in dfs_uplk[0].keys():
        dfs_uplk[0][schm] = pd.concat([dfs_uplk[0][schm], dfs_uplk[i][schm]], ignore_index=True)
dfs_uplk = dfs_uplk[0]

## B3+B7

### Uplink Loss

In [131]:
### B3
# specify a scheme
schemes = ['B3','B7']
schm = schemes[0]
df = dfs_uplk['B3+B7']

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B3 = table[table[f'lost_{schm}'] != 0]


### B8
# specify a scheme
# schemes = ['B3','B8']
schm = schemes[1]

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B7 = table[table[f'lost_{schm}'] != 0]


### B3+B8
# specify a scheme
# schemes = ['B3','B7']
schm1, schm2 = schemes[0], schemes[1]

# count loss
key = f'{schm1}+{schm2}'
table = df.loc[df[f'lost_{key}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{key}'})
table[f'lost_{key}'] = table[f'lost_{key}'].astype('Int16')

key1 = f'{schm1}+^{schm2}'
table1 = df.loc[df[f'lost_{key1}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key1}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key1}'] = table[f'lost_{key1}'].astype('Int16')

key2 = f'^{schm1}+{schm2}'
table1 = df.loc[df[f'lost_{key2}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key2}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key2}'] = table[f'lost_{key2}'].astype('Int16')

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{key}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)

# calculate loss rate
table[f'lorate(%)_{key}'] = round(table[f'lost_{key}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key1}'] = round(table[f'lost_{key1}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key2}'] = round(table[f'lost_{key2}'] / table[f'total_{key}'] * 100, 3)

# show table 1
# table_B3_B8 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B3_B7 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key}'] != 0),
                  [f'ho_type0_{schm1}', f'ho_type0_{schm2}', f'lost_{key}', f'lost_{key1}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key1}']]

# show table 2
# table_B8_B3 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B7_B3 = table.loc[(table[f'lost_{key2}'] != 0) | (table[f'lost_{key}'] != 0),
                  [f'ho_type0_{schm2}', f'ho_type0_{schm1}', f'lost_{key}', f'lost_{key2}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key2}']] \
                    .sort_values([f'ho_type0_{schm2}', f'ho_type0_{schm1}'])
                    
display(table_B3)
display(table_B3_B7)
display(table_B7)
display(table_B7_B3)

,ho_type0_B3,lost_B3,total_B3,lorate(%)_B3
0,stable,119,746769,0.016
1,LTE_HO,5,2547,0.196
2,MN_HO,48,45445,0.106
3,SN_HO,10,57178,0.017
5,SN_Rel,8,5066,0.158
6,SN_Setup,81,11279,0.718
17,MCG_Failure_handoverFailure (1),1,812,0.123


,ho_type0_B3,ho_type0_B7,lost_B3+B7,lost_B3+^B7,total_B3+B7,lorate(%)_B3+B7,lorate(%)_B3+^B7


,ho_type0_B7,lost_B7,total_B7,lorate(%)_B7
0,stable,24,721822,0.003
2,MN_HO,86,59906,0.144
3,SN_HO,7,62872,0.011
5,SN_Rel,7,3867,0.181
6,SN_Setup,160,9414,1.7
17,MCG_Failure_handoverFailure (1),1,765,0.131
20,NAS_Recovery_otherFailure (2),117,1828,6.4


,ho_type0_B7,ho_type0_B3,lost_B3+B7,lost_^B3+B7,total_B3+B7,lorate(%)_B3+B7,lorate(%)_^B3+B7


### Downlink Loss

In [132]:
### B3
# specify a scheme
schemes = ['B3','B7']
schm = schemes[0]
df = dfs_dnlk['B3+B7']

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B3 = table[table[f'lost_{schm}'] != 0]


### B8
# specify a scheme
# schemes = ['B3','B8']
schm = schemes[1]

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B7 = table[table[f'lost_{schm}'] != 0]


### B3+B8
# specify a scheme
# schemes = ['B3','B7']
schm1, schm2 = schemes[0], schemes[1]

# count loss
key = f'{schm1}+{schm2}'
table = df.loc[df[f'lost_{key}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{key}'})
table[f'lost_{key}'] = table[f'lost_{key}'].astype('Int16')

key1 = f'{schm1}+^{schm2}'
table1 = df.loc[df[f'lost_{key1}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key1}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key1}'] = table[f'lost_{key1}'].astype('Int16')

key2 = f'^{schm1}+{schm2}'
table1 = df.loc[df[f'lost_{key2}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key2}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key2}'] = table[f'lost_{key2}'].astype('Int16')

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{key}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)

# calculate loss rate
table[f'lorate(%)_{key}'] = round(table[f'lost_{key}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key1}'] = round(table[f'lost_{key1}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key2}'] = round(table[f'lost_{key2}'] / table[f'total_{key}'] * 100, 3)

# show table 1
# table_B3_B8 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B3_B7 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key}'] != 0),
                  [f'ho_type0_{schm1}', f'ho_type0_{schm2}', f'lost_{key}', f'lost_{key1}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key1}']]

# show table 2
# table_B8_B3 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B7_B3 = table.loc[(table[f'lost_{key2}'] != 0) | (table[f'lost_{key}'] != 0),
                  [f'ho_type0_{schm2}', f'ho_type0_{schm1}', f'lost_{key}', f'lost_{key2}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key2}']] \
                    .sort_values([f'ho_type0_{schm2}', f'ho_type0_{schm1}'])
                    
display(table_B3)
display(table_B3_B7)
display(table_B7)
display(table_B7_B3)

,ho_type0_B3,lost_B3,total_B3,lorate(%)_B3
0,stable,3,746782,0.0
1,LTE_HO,23,2617,0.879
2,MN_HO,336,45399,0.74
3,SN_HO,31,56981,0.054
6,SN_Setup,334,11562,2.889
17,MCG_Failure_handoverFailure (1),541,789,68.568


,ho_type0_B3,ho_type0_B7,lost_B3+B7,lost_B3+^B7,total_B3+B7,lorate(%)_B3+B7,lorate(%)_B3+^B7
0,stable,SN_HO,0,3,26918,0.0,0.011
7,LTE_HO,stable,0,22,808,0.0,2.723
8,LTE_HO,MN_HO,0,1,1006,0.0,0.099
12,MN_HO,NAS_Recovery_otherFailure (2),332,0,1011,32.839,0.0
13,MN_HO,MN_HO,0,2,12632,0.0,0.016
16,MN_HO,stable,0,2,20848,0.0,0.01
20,SN_HO,stable,0,1,21921,0.0,0.005
21,SN_HO,SN_HO,0,30,22935,0.0,0.131
30,SN_Setup,SN_HO,0,146,1861,0.0,7.845
31,SN_Setup,stable,0,119,4784,0.0,2.487


,ho_type0_B7,lost_B7,total_B7,lorate(%)_B7
0,stable,372,721727,0.052
1,LTE_HO,206,2613,7.884
2,MN_HO,1580,60361,2.618
3,SN_HO,119,63076,0.189
5,SN_Rel,8,3604,0.222
6,SN_Setup,1163,9277,12.536
17,MCG_Failure_handoverFailure (1),760,760,100.0
20,NAS_Recovery_otherFailure (2),1171,1809,64.732


,ho_type0_B7,ho_type0_B3,lost_B3+B7,lost_^B3+B7,total_B3+B7,lorate(%)_B3+B7,lorate(%)_^B3+B7
1,stable,stable,0,12,671066,0.0,0.002
16,stable,MN_HO,0,360,20848,0.0,1.727
2,LTE_HO,stable,0,206,1683,0.0,12.24
5,MN_HO,stable,0,889,36546,0.0,2.433
13,MN_HO,MN_HO,0,200,12632,0.0,1.583
19,MN_HO,SN_HO,0,180,7440,0.0,2.419
29,MN_HO,SN_Rel,0,149,1469,0.0,10.143
34,MN_HO,SN_Setup,0,162,1268,0.0,12.776
14,SN_HO,MN_HO,0,8,8657,0.0,0.092
21,SN_HO,SN_HO,0,111,22935,0.0,0.484


## B3+B8

### Uplink Loss

In [133]:
### B3
# specify a scheme
schemes = ['B3','B8']
schm = schemes[0]
df = dfs_uplk['B3+B8']

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B3 = table[table[f'lost_{schm}'] != 0]


### B8
# specify a scheme
# schemes = ['B3','B8']
schm = schemes[1]

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B8 = table[table[f'lost_{schm}'] != 0]


### B3+B8
# specify a scheme
# schemes = ['B3','B8']
schm1, schm2 = schemes[0], schemes[1]

# count loss
key = f'{schm1}+{schm2}'
table = df.loc[df[f'lost_{key}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{key}'})
table[f'lost_{key}'] = table[f'lost_{key}'].astype('Int16')

key1 = f'{schm1}+^{schm2}'
table1 = df.loc[df[f'lost_{key1}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key1}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key1}'] = table[f'lost_{key1}'].astype('Int16')

key2 = f'^{schm1}+{schm2}'
table1 = df.loc[df[f'lost_{key2}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key2}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key2}'] = table[f'lost_{key2}'].astype('Int16')

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{key}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)

# calculate loss rate
table[f'lorate(%)_{key}'] = round(table[f'lost_{key}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key1}'] = round(table[f'lost_{key1}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key2}'] = round(table[f'lost_{key2}'] / table[f'total_{key}'] * 100, 3)

# show table 1
# table_B3_B8 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B3_B8 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key}'] != 0),
                  [f'ho_type0_{schm1}', f'ho_type0_{schm2}', f'lost_{key}', f'lost_{key1}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key1}']]

# show table 2
# table_B8_B3 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B8_B3 = table.loc[(table[f'lost_{key2}'] != 0) | (table[f'lost_{key}'] != 0),
                  [f'ho_type0_{schm2}', f'ho_type0_{schm1}', f'lost_{key}', f'lost_{key2}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key2}']] \
                    .sort_values([f'ho_type0_{schm2}', f'ho_type0_{schm1}'])
                    
display(table_B3)
display(table_B3_B8)
display(table_B8)
display(table_B8_B3)

,ho_type0_B3,lost_B3,total_B3,lorate(%)_B3
0,stable,119,746769,0.016
1,LTE_HO,5,2547,0.196
2,MN_HO,48,45445,0.106
3,SN_HO,10,57178,0.017
5,SN_Rel,8,5066,0.158
6,SN_Setup,81,11279,0.718
17,MCG_Failure_handoverFailure (1),1,812,0.123


,ho_type0_B3,ho_type0_B8,lost_B3+B8,lost_B3+^B8,total_B3+B8,lorate(%)_B3+B8,lorate(%)_B3+^B8


,ho_type0_B8,lost_B8,total_B8,lorate(%)_B8
0,stable,42,698674,0.006
2,MN_HO,170,90789,0.187
3,SN_HO,14,62367,0.022
5,SN_Rel,1,3975,0.025
6,SN_Setup,145,9600,1.51


,ho_type0_B8,ho_type0_B3,lost_B3+B8,lost_^B3+B8,total_B3+B8,lorate(%)_B3+B8,lorate(%)_^B3+B8


### Downlink Loss

In [134]:
### B3
# specify a scheme
schemes = ['B3','B8']
schm = schemes[0]
df = dfs_dnlk['B3+B8']

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B3 = table[table[f'lost_{schm}'] != 0]


### B8
# specify a scheme
# schemes = ['B3','B8']
schm = schemes[1]

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B8 = table[table[f'lost_{schm}'] != 0]


### B3+B8
# specify a scheme
# schemes = ['B3','B8']
schm1, schm2 = schemes[0], schemes[1]

# count loss
key = f'{schm1}+{schm2}'
table = df.loc[df[f'lost_{key}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{key}'})
table[f'lost_{key}'] = table[f'lost_{key}'].astype('Int16')

key1 = f'{schm1}+^{schm2}'
table1 = df.loc[df[f'lost_{key1}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key1}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key1}'] = table[f'lost_{key1}'].astype('Int16')

key2 = f'^{schm1}+{schm2}'
table1 = df.loc[df[f'lost_{key2}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key2}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key2}'] = table[f'lost_{key2}'].astype('Int16')

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{key}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)

# calculate loss rate
table[f'lorate(%)_{key}'] = round(table[f'lost_{key}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key1}'] = round(table[f'lost_{key1}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key2}'] = round(table[f'lost_{key2}'] / table[f'total_{key}'] * 100, 3)

# show table 1
# table_B3_B8 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B3_B8 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key}'] != 0),
                  [f'ho_type0_{schm1}', f'ho_type0_{schm2}', f'lost_{key}', f'lost_{key1}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key1}']]

# show table 2
# table_B8_B3 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B8_B3 = table.loc[(table[f'lost_{key2}'] != 0) | (table[f'lost_{key}'] != 0),
                  [f'ho_type0_{schm2}', f'ho_type0_{schm1}', f'lost_{key}', f'lost_{key2}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key2}']] \
                    .sort_values([f'ho_type0_{schm2}', f'ho_type0_{schm1}'])
                    
display(table_B3)
display(table_B3_B8)
display(table_B8)
display(table_B8_B3)

,ho_type0_B3,lost_B3,total_B3,lorate(%)_B3
0,stable,3,746782,0.0
1,LTE_HO,23,2617,0.879
2,MN_HO,336,45399,0.74
3,SN_HO,31,56981,0.054
6,SN_Setup,334,11562,2.889
17,MCG_Failure_handoverFailure (1),541,789,68.568


,ho_type0_B3,ho_type0_B8,lost_B3+B8,lost_B3+^B8,total_B3+B8,lorate(%)_B3+B8,lorate(%)_B3+^B8
0,stable,MN_HO,0,2,69932,0.0,0.003
1,stable,SN_HO,0,1,25407,0.0,0.004
10,LTE_HO,MN_HO,0,23,378,0.0,6.085
15,MN_HO,SN_HO,0,3,10268,0.0,0.029
16,MN_HO,stable,0,333,23978,0.0,1.389
19,SN_HO,stable,0,3,22131,0.0,0.014
20,SN_HO,MN_HO,0,28,9240,0.0,0.303
29,SN_Setup,stable,0,117,5144,0.0,2.274
30,SN_Setup,SN_HO,0,71,2497,0.0,2.843
31,SN_Setup,SN_Setup,0,18,1639,0.0,1.098


,ho_type0_B8,lost_B8,total_B8,lorate(%)_B8
0,stable,196,699007,0.028
2,MN_HO,480,90636,0.53
3,SN_HO,3,62684,0.005
5,SN_Rel,1,3875,0.026
6,SN_Setup,904,9776,9.247


,ho_type0_B8,ho_type0_B3,lost_B3+B8,lost_^B3+B8,total_B3+B8,lorate(%)_B3+B8,lorate(%)_^B3+B8
3,stable,stable,0,184,644594,0.0,0.029
29,stable,SN_Setup,0,12,5144,0.0,0.233
0,MN_HO,stable,0,355,69932,0.0,0.508
10,MN_HO,LTE_HO,0,1,378,0.0,0.265
12,MN_HO,MN_HO,0,83,6810,0.0,1.219
20,MN_HO,SN_HO,0,10,9240,0.0,0.108
33,MN_HO,SN_Setup,0,30,1450,0.0,2.069
1,SN_HO,stable,0,1,25407,0.0,0.004
15,SN_HO,MN_HO,0,2,10268,0.0,0.019
11,SN_Rel,MN_HO,0,1,1690,0.0,0.059


## B7+B8

### Uplink Loss

In [135]:
### B3
# specify a scheme
schemes = ['B7','B8']
schm = schemes[0]
df = dfs_uplk['B7+B8']

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B7 = table[table[f'lost_{schm}'] != 0]


### B8
# specify a scheme
# schemes = ['B3','B8']
schm = schemes[1]

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B8 = table[table[f'lost_{schm}'] != 0]


### B3+B8
# specify a scheme
# schemes = ['B3','B7']
schm1, schm2 = schemes[0], schemes[1]

# count loss
key = f'{schm1}+{schm2}'
table = df.loc[df[f'lost_{key}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{key}'})
table[f'lost_{key}'] = table[f'lost_{key}'].astype('Int16')

key1 = f'{schm1}+^{schm2}'
table1 = df.loc[df[f'lost_{key1}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key1}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key1}'] = table[f'lost_{key1}'].astype('Int16')

key2 = f'^{schm1}+{schm2}'
table1 = df.loc[df[f'lost_{key2}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key2}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key2}'] = table[f'lost_{key2}'].astype('Int16')

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{key}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)

# calculate loss rate
table[f'lorate(%)_{key}'] = round(table[f'lost_{key}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key1}'] = round(table[f'lost_{key1}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key2}'] = round(table[f'lost_{key2}'] / table[f'total_{key}'] * 100, 3)

# show table 1
# table_B3_B8 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B7_B8 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key}'] != 0),
                  [f'ho_type0_{schm1}', f'ho_type0_{schm2}', f'lost_{key}', f'lost_{key1}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key1}']]

# show table 2
# table_B8_B3 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B8_B7 = table.loc[(table[f'lost_{key2}'] != 0) | (table[f'lost_{key}'] != 0),
                  [f'ho_type0_{schm2}', f'ho_type0_{schm1}', f'lost_{key}', f'lost_{key2}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key2}']] \
                    .sort_values([f'ho_type0_{schm2}', f'ho_type0_{schm1}'])
                    
display(table_B7)
display(table_B7_B8)
display(table_B8)
display(table_B8_B7)

,ho_type0_B7,lost_B7,total_B7,lorate(%)_B7
0,stable,24,721822,0.003
2,MN_HO,86,59906,0.144
3,SN_HO,7,62872,0.011
5,SN_Rel,7,3867,0.181
6,SN_Setup,160,9414,1.7
17,MCG_Failure_handoverFailure (1),1,765,0.131
20,NAS_Recovery_otherFailure (2),117,1828,6.4


,ho_type0_B7,ho_type0_B8,lost_B7+B8,lost_B7+^B8,total_B7+B8,lorate(%)_B7+B8,lorate(%)_B7+^B8


,ho_type0_B8,lost_B8,total_B8,lorate(%)_B8
0,stable,42,698674,0.006
2,MN_HO,170,90789,0.187
3,SN_HO,14,62367,0.022
5,SN_Rel,1,3975,0.025
6,SN_Setup,145,9600,1.51


,ho_type0_B8,ho_type0_B7,lost_B7+B8,lost_^B7+B8,total_B7+B8,lorate(%)_B7+B8,lorate(%)_^B7+B8


### Downlink Loss

In [136]:
### B3
# specify a scheme
schemes = ['B7','B8']
schm = schemes[0]
df = dfs_dnlk['B7+B8']

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B7 = table[table[f'lost_{schm}'] != 0]


### B8
# specify a scheme
# schemes = ['B3','B8']
schm = schemes[1]

# count loss
table = df.loc[df[f'lost_{schm}'], [f'ho_type0_{schm}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{schm}'})

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{schm}'})

# merge table
table = table.merge(table1, on=[f'ho_type0_{schm}'], how='outer') \
    .sort_values([f'ho_type0_{schm}']).reset_index(drop=True).fillna(0)
table[f'lost_{schm}'] = table[f'lost_{schm}'].astype('Int16')

# calculate loss rate
table[f'lorate(%)_{schm}'] = round(table[f'lost_{schm}'] / table[f'total_{schm}'] * 100, 3)

# show table
table_B8 = table[table[f'lost_{schm}'] != 0]


### B3+B8
# specify a scheme
# schemes = ['B3','B7']
schm1, schm2 = schemes[0], schemes[1]

# count loss
key = f'{schm1}+{schm2}'
table = df.loc[df[f'lost_{key}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table = pd.DataFrame(table).reset_index().rename(columns={0:f'lost_{key}'})
table[f'lost_{key}'] = table[f'lost_{key}'].astype('Int16')

key1 = f'{schm1}+^{schm2}'
table1 = df.loc[df[f'lost_{key1}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key1}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key1}'] = table[f'lost_{key1}'].astype('Int16')

key2 = f'^{schm1}+{schm2}'
table1 = df.loc[df[f'lost_{key2}'], [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'lost_{key2}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)
table[f'lost_{key2}'] = table[f'lost_{key2}'].astype('Int16')

# count total packet
table1 = df.loc[:, [f'ho_type0_{schm1}', f'ho_type0_{schm2}']].value_counts()
table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'total_{key}'})
table = table.merge(table1, on=[f'ho_type0_{schm1}', f'ho_type0_{schm2}'], how='outer') \
    .sort_values([f'ho_type0_{schm1}']).reset_index(drop=True).fillna(0)

# calculate loss rate
table[f'lorate(%)_{key}'] = round(table[f'lost_{key}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key1}'] = round(table[f'lost_{key1}'] / table[f'total_{key}'] * 100, 3)
table[f'lorate(%)_{key2}'] = round(table[f'lost_{key2}'] / table[f'total_{key}'] * 100, 3)

# show table 1
# table_B3_B8 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B7_B8 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key}'] != 0),
                  [f'ho_type0_{schm1}', f'ho_type0_{schm2}', f'lost_{key}', f'lost_{key1}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key1}']]

# show table 2
# table_B8_B3 = table.loc[(table[f'lost_{key1}'] != 0) | (table[f'lost_{key2}'] != 0),
table_B8_B7 = table.loc[(table[f'lost_{key2}'] != 0) | (table[f'lost_{key}'] != 0),
                  [f'ho_type0_{schm2}', f'ho_type0_{schm1}', f'lost_{key}', f'lost_{key2}', f'total_{key}', f'lorate(%)_{key}', f'lorate(%)_{key2}']] \
                    .sort_values([f'ho_type0_{schm2}', f'ho_type0_{schm1}'])
                    
display(table_B7)
display(table_B7_B8)
display(table_B8)
display(table_B8_B7)

,ho_type0_B7,lost_B7,total_B7,lorate(%)_B7
0,stable,372,721727,0.052
1,LTE_HO,206,2613,7.884
2,MN_HO,1580,60361,2.618
3,SN_HO,119,63076,0.189
5,SN_Rel,8,3604,0.222
6,SN_Setup,1163,9277,12.536
17,MCG_Failure_handoverFailure (1),760,760,100.0
20,NAS_Recovery_otherFailure (2),1171,1809,64.732


,ho_type0_B7,ho_type0_B8,lost_B7+B8,lost_B7+^B8,total_B7+B8,lorate(%)_B7+B8,lorate(%)_B7+^B8
0,stable,SN_HO,0,184,21587,0.0,0.852
1,stable,stable,0,188,633339,0.0,0.03
8,LTE_HO,SN_HO,0,36,808,0.0,4.455
9,LTE_HO,stable,0,170,1452,0.0,11.708
10,MN_HO,MN_HO,0,114,16814,0.0,0.678
12,MN_HO,SN_Rel,0,149,1912,0.0,7.793
14,MN_HO,stable,0,1137,32256,0.0,3.525
16,SN_HO,SN_Setup,0,1,1839,0.0,0.054
17,SN_HO,SN_HO,0,7,28975,0.0,0.024
18,SN_HO,MN_HO,0,3,9557,0.0,0.031


,ho_type0_B8,lost_B8,total_B8,lorate(%)_B8
0,stable,196,699007,0.028
2,MN_HO,480,90636,0.53
3,SN_HO,3,62684,0.005
5,SN_Rel,1,3875,0.026
6,SN_Setup,904,9776,9.247


,ho_type0_B8,ho_type0_B7,lost_B7+B8,lost_^B7+B8,total_B7+B8,lorate(%)_B7+B8,lorate(%)_^B7+B8
1,stable,stable,0,83,633339,0.0,0.013
14,stable,MN_HO,0,58,32256,0.0,0.18
21,stable,SN_HO,0,7,20516,0.0,0.034
31,stable,SN_Setup,0,48,4737,0.0,1.013
2,MN_HO,stable,0,96,59205,0.0,0.162
10,MN_HO,MN_HO,0,193,16814,0.0,1.148
32,MN_HO,SN_Setup,0,77,1644,0.0,4.684
0,SN_HO,stable,0,2,21587,0.0,0.009
17,SN_HO,SN_HO,0,1,28975,0.0,0.003
12,SN_Rel,MN_HO,0,1,1912,0.0,0.052
